# Feature Engineering

#### Based on the insights generated from the exploratory data analysis, we will built the features that would work for the dataset and our usecase objective.

#### Features created in different tiers considering latency & complexity:
* Tier 1: Simple features & Fast calculation
* Tier 2: Signal Quality features & Moderate
* Tier 3: Advanced Features & Slower
* Power System Specific features

### Steps for creating the features:
* __Subset Selection__: 600K out of 800K measurements (75%) per signal and 5K/8.7K signals are selected to be memory and time efficient 
* __Chunk Signal__: For Feature calculation, each 600K measurements per signal is divded  into chunks of 100 measurements for easier calculation
* __Feature creation__ :
	* Features across all tiers are created at chunk level
	* Features across all chunks are aggregated back to signal level using statistical measures – std,mean etc.
* __Preprocessing & Feature Selection__ :
	* Handling missing /infinite values and outliers
	* Selecting High Variance Features
	* Scaling with RobustScaler in presence of outliers

In [38]:
import numpy as np
import pandas as pd
from scipy import signal, stats
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks, periodogram, welch
import pywt
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

In [39]:
DATA_PATH = r'./data/vsb-power-line-fault-detection/'
FEATURE_PATH = r"./features//"
CHUNK_SIZE = 1000
SAMPLE_SIZE=1000

In [40]:
metadata_df  = pd.read_csv(f"{DATA_PATH}/metadata_train.csv")

In [41]:
%%time 
train_path = f"{DATA_PATH}/train.parquet"
signals_df = pd.read_parquet(train_path, engine='fastparquet')

CPU times: total: 53.2 s
Wall time: 59.9 s


### Modules for feature Creation

#### 1. Basic Features (Fastest)

In [42]:
def extract_basic_features(signal_chunk):
    """
    Extract Basic features: Faster statistical features (< 1ms)
    These are the most discriminative and fastest to compute
    
    Args:
        signal_chunk: 1D array of signal values
        
    Returns:
        dict: Dictionary of Tier 1 features
    """
    features = {}
    
    # Core statistical features - highest discriminative power from EDA
    features['std'] = np.std(signal_chunk)  # Most discriminative single feature
    features['rms'] = np.sqrt(np.mean(signal_chunk**2))  # Power system standard
    features['mean_abs'] = np.mean(np.abs(signal_chunk))
    
    # Amplitude-based features - clear separation observed in EDA
    features['min_val'] = np.min(signal_chunk)
    features['max_val'] = np.max(signal_chunk)
    features['peak_to_peak'] = features['max_val'] - features['min_val']
    
    # Power system specific ratios - efficient fault indicators
    features['crest_factor'] = features['max_val'] / features['rms'] if features['rms'] > 1e-10 else 0
    features['form_factor'] = features['rms'] / features['mean_abs'] if features['mean_abs'] > 1e-10 else 0
    
    # Deviation measures - fault signals showed higher variability
    features['mean_abs_deviation'] = np.mean(np.abs(signal_chunk - np.mean(signal_chunk)))
    features['variance'] = np.var(signal_chunk)
    
    return features

#### 2. Signal Quality Feature (Moderate Speed)

In [43]:
def extract_signal_quality_features(signal_chunk):
    """
    Extract Tier 2 features: Signal quality features (1-5ms)
    Moderate computational cost, good discriminative power
    
    Args:
        signal_chunk: 1D array of signal values
        
    Returns:
        dict: Dictionary of Tier 2 features
    """
    features = {}
    
    # Zero crossing analysis - fault signals showed irregular patterns
    zero_crossings = np.where(np.diff(np.sign(signal_chunk)))[0]
    features['zero_crossing_rate'] = len(zero_crossings) / len(signal_chunk)
    
    # Peak detection - fault signals had distinctive peak patterns
    mean_val = np.mean(signal_chunk)
    std_val = np.std(signal_chunk)
    threshold = mean_val + 2 * std_val  # Dynamic threshold
    
    peaks_positive = np.where(signal_chunk > threshold)[0]
    peaks_negative = np.where(signal_chunk < (mean_val - 2 * std_val))[0]
    
    features['peak_count'] = len(peaks_positive) + len(peaks_negative)
    features['peak_density'] = features['peak_count'] / len(signal_chunk)
    
    # Signal envelope analysis - sliding window variability patterns
    window_size = min(100, len(signal_chunk) // 10)
    if window_size > 1:
        envelope = []
        for i in range(0, len(signal_chunk) - window_size + 1, window_size // 2):
            window = signal_chunk[i:i + window_size]
            envelope.append(np.max(np.abs(window)))
        
        features['envelope_mean'] = np.mean(envelope)
        features['envelope_std'] = np.std(envelope)
        features['envelope_variation'] = features['envelope_std'] / features['envelope_mean'] if features['envelope_mean'] > 1e-10 else 0
    else:
        features['envelope_mean'] = np.max(np.abs(signal_chunk))
        features['envelope_std'] = 0
        features['envelope_variation'] = 0
    
    # Simple frequency domain feature - high frequency content ratio
    fft_vals = np.abs(np.fft.fft(signal_chunk))
    n_samples = len(fft_vals) // 2  # Only positive frequencies
    
    # Split into low and high frequency bands
    split_point = n_samples // 4  # 25% split point
    low_freq_energy = np.sum(fft_vals[:split_point]**2)
    high_freq_energy = np.sum(fft_vals[split_point:n_samples]**2)
    total_energy = low_freq_energy + high_freq_energy
    
    features['high_freq_ratio'] = high_freq_energy / total_energy if total_energy > 1e-10 else 0
    
    return features

#### 3. Advanced Features (Comparitively slower)

In [44]:
def extract_advanced_features(signal_chunk,sampling_rate):
    """
    Extract Tier 3 features: Advanced features (5-20ms)
    Higher computational cost, use only when accuracy is critical
    
    Args:
        signal_chunk: 1D array of signal values
        sampling_rate: rate of sampling
        
    Returns:
        dict: Dictionary of Tier 3 features
    """
    
    features = {}
    
    # Spectral features - frequency domain analysis showed clear patterns
    fft_vals = np.fft.fft(signal_chunk)
    fft_freq = np.fft.fftfreq(len(signal_chunk), 1/sampling_rate)
    
    # Get positive frequencies only
    positive_freq_mask = fft_freq > 0
    frequencies = fft_freq[positive_freq_mask]
    magnitudes = np.abs(fft_vals[positive_freq_mask])
    power = magnitudes**2
    
    if np.sum(magnitudes) > 1e-10:
        # Spectral centroid - frequency center of mass
        features['spectral_centroid'] = np.sum(frequencies * magnitudes) / np.sum(magnitudes)
        
        # Dominant frequency features
        dominant_freq_idx = np.argmax(magnitudes)
        features['dominant_frequency'] = frequencies[dominant_freq_idx]
        features['dominant_magnitude'] = magnitudes[dominant_freq_idx]
        features['dominant_power_ratio'] = magnitudes[dominant_freq_idx]**2 / np.sum(power)
    else:
        features['spectral_centroid'] = 0
        features['dominant_frequency'] = 0
        features['dominant_magnitude'] = 0
        features['dominant_power_ratio'] = 0
    
    # Simple wavelet analysis - 3 levels only for efficiency
    try:
        coeffs = pywt.wavedec(signal_chunk, 'db4', level=3)
        
        # Energy distribution across levels
        total_energy = sum([np.sum(coeff**2) for coeff in coeffs])
        
        if total_energy > 1e-10:
            for i, coeff in enumerate(coeffs):
                level_name = 'approx' if i == 0 else f'detail_{i}'
                features[f'wavelet_{level_name}_energy_ratio'] = np.sum(coeff**2) / total_energy
        else:
            for i in range(4):  # 3 levels + approximation
                level_name = 'approx' if i == 0 else f'detail_{i}'
                features[f'wavelet_{level_name}_energy_ratio'] = 0
                
    except:
        # Fallback if wavelet fails
        for i in range(4):
            level_name = 'approx' if i == 0 else f'detail_{i}'
            features[f'wavelet_{level_name}_energy_ratio'] = 0
    
    # Autocorrelation feature 
    if len(signal_chunk) > 10:
        autocorr = np.correlate(signal_chunk, signal_chunk, mode='full')
        autocorr = autocorr[len(autocorr)//2:]
        autocorr = autocorr / autocorr[0] if autocorr[0] != 0 else autocorr
        
        # Find first minimum (indicates periodicity)
        if len(autocorr) > 2:
            features['autocorr_first_min'] = np.argmin(autocorr[1:]) + 1
        else:
            features['autocorr_first_min'] = 0
    else:
        features['autocorr_first_min'] = 0
    
    return features

#### 4. Power System specific Features

In [45]:
def extract_power_system_specific_features(signal_chunk,sampling_rate):
    """
    Extract power system specific features optimized for grid applications
    
    Args:
        signal_chunk: 1D array of signal values
        
    Returns:
        dict: Dictionary of power system specific features
    """
    features = {}
    
    # Simplified THD estimation for real-time applications
    fft_vals = np.abs(np.fft.fft(signal_chunk))
    fft_freq = np.fft.fftfreq(len(signal_chunk), 1/sampling_rate)
    
    # Look for fundamental frequency in reasonable power system range (40-70 Hz)
    power_freq_mask = (fft_freq >= 40) & (fft_freq <= 70)
    
    if np.any(power_freq_mask):
        fundamental_idx = np.argmax(fft_vals[power_freq_mask])
        fundamental_freq = fft_freq[power_freq_mask][fundamental_idx]
        fundamental_magnitude = fft_vals[power_freq_mask][fundamental_idx]
        
        # Simple harmonic estimation - 2nd and 3rd harmonics only for speed
        harmonic_energy = 0
        for harmonic in [2, 3]:  # Only check key harmonics
            harmonic_freq = fundamental_freq * harmonic
            harmonic_idx = np.argmin(np.abs(fft_freq - harmonic_freq))
            if harmonic_idx < len(fft_vals):
                harmonic_energy += fft_vals[harmonic_idx]**2
        
        features['thd_estimate'] = np.sqrt(harmonic_energy) / fundamental_magnitude if fundamental_magnitude > 1e-10 else 0
        features['fundamental_frequency'] = fundamental_freq
    else:
        features['thd_estimate'] = 0
        features['fundamental_frequency'] = 0
    
    # Transient detection - sudden amplitude changes
    if len(signal_chunk) > 1:
        diff_signal = np.diff(signal_chunk)
        features['transient_index'] = np.std(diff_signal) / np.std(signal_chunk) if np.std(signal_chunk) > 1e-10 else 0
        features['max_gradient'] = np.max(np.abs(diff_signal))
    else:
        features['transient_index'] = 0
        features['max_gradient'] = 0
    
    # Voltage sag/swell detection (simplified)
    features['amplitude_variation'] = (np.max(signal_chunk) - np.min(signal_chunk)) / np.std(signal_chunk) if np.std(signal_chunk) > 1e-10 else 0
    
    return features

#### Helper function for chunking signals

In [46]:
def chunk_signal(signal,chunk_size):
    """
    Divide signal into chunks for feature extraction
    
    Args:
        signal: 1D array of signal values
        
    Returns:
        list: List of signal chunks
    """
    chunks = []
    n_chunks = len(signal) // chunk_size
    
    for i in range(n_chunks):
        start_idx = i * chunk_size
        end_idx = start_idx + chunk_size
        chunks.append(signal[start_idx:end_idx])
    
    # Handle remaining samples
    if len(signal) % chunk_size != 0:
        chunks.append(signal[n_chunks * chunk_size:])
    
    return chunks

### Data Preparation 

In [47]:
signal_columns = signals_df.columns

##### Creating Subset of Measurements per Signal (60K/80K) & subset of samples (5K/8.7K)

In [48]:
subset_signals =signals_df.loc[10001:70000,:'4999'].copy()
subset_signals.shape

(60000, 5000)

In [49]:
subset_metadata = metadata_df.loc[:4999,:].copy()
subset_metadata.shape

(5000, 4)

In [50]:
#Target distribution in subset
subset_metadata['target'].value_counts()

target
0    4668
1     332
Name: count, dtype: int64

### Feature Engineering at Chunk Level

In [51]:
%%time 
all_features = []
signal_columns = subset_signals.columns
for i, signal_id in enumerate(signal_columns):
    if  i % 100 == 0:
        print(f"Processing signal {i+1}/{len(signal_columns)}")

    
    signal = subset_signals[signal_id].values

    # Divide signal into chunks
    chunks = chunk_signal(signal,CHUNK_SIZE)
    
    feature_list = []
    
    for chunk_idx, chunk in enumerate(chunks):
        if len(chunk) < 10:  # Skip very small chunks
            continue
            
        chunk_features = {}
        chunk_features['signal_id'] = signal_id if signal_id is not None else 0
        chunk_features['chunk_id'] = chunk_idx
        
        # Extract Basic (fastest, most important)
        chunk_features.update(extract_basic_features(chunk))
        
        # Extract signal quality features
        chunk_features.update(extract_signal_quality_features(chunk))
        
        # Extract advanced features
        chunk_features.update(extract_advanced_features(chunk,SAMPLE_SIZE))
        
        # Power system specific features 
        chunk_features.update(extract_power_system_specific_features(chunk,SAMPLE_SIZE))
        
        feature_list.append(chunk_features)         
    
    
    signal_features = pd.DataFrame(feature_list)
    
    # Add metadata if available
    if subset_metadata is not None and int(signal_id) in subset_metadata['signal_id'].values:
        meta_row = subset_metadata[subset_metadata['signal_id'] == int(signal_id)].iloc[0]
        for _, row in signal_features.iterrows():
            row_dict = row.to_dict()
            for col in subset_metadata.columns:
                if col != 'signal_id':
                    row_dict[col] = meta_row[col]
            all_features.append(row_dict)
    else:
        all_features.extend(signal_features.to_dict('records'))

feature_df = pd.DataFrame(all_features)

Processing signal 1/5000
Processing signal 101/5000
Processing signal 201/5000
Processing signal 301/5000
Processing signal 401/5000
Processing signal 501/5000
Processing signal 601/5000
Processing signal 701/5000
Processing signal 801/5000
Processing signal 901/5000
Processing signal 1001/5000
Processing signal 1101/5000
Processing signal 1201/5000
Processing signal 1301/5000
Processing signal 1401/5000
Processing signal 1501/5000
Processing signal 1601/5000
Processing signal 1701/5000
Processing signal 1801/5000
Processing signal 1901/5000
Processing signal 2001/5000
Processing signal 2101/5000
Processing signal 2201/5000
Processing signal 2301/5000
Processing signal 2401/5000
Processing signal 2501/5000
Processing signal 2601/5000
Processing signal 2701/5000
Processing signal 2801/5000
Processing signal 2901/5000
Processing signal 3001/5000
Processing signal 3101/5000
Processing signal 3201/5000
Processing signal 3301/5000
Processing signal 3401/5000
Processing signal 3501/5000
Proc

In [52]:
feature_df.shape

(300000, 36)

In [53]:
feature_df.head()

,signal_id,chunk_id,std,rms,mean_abs,min_val,max_val,peak_to_peak,crest_factor,form_factor,...,wavelet_detail_3_energy_ratio,autocorr_first_min,thd_estimate,fundamental_frequency,transient_index,max_gradient,amplitude_variation,id_measurement,phase,target
0,0,0,0.940083,5.305280,16.838,14,20,6,3.769829,0.315078,...,0.001958,48,0.334519,40.0,1.580357,5,6.382415,0,0,0
1,0,1,0.913902,4.020199,16.472,13,19,6,4.726134,0.244063,...,0.001858,57,0.344680,40.0,1.540460,5,6.565259,0,0,0
2,0,2,0.971391,2.788548,16.220,13,20,7,7.172191,0.171920,...,0.002370,219,0.406761,40.0,1.601921,5,7.206163,0,0,0
3,0,3,0.873531,NaN,15.712,12,18,6,0.000000,NaN,...,0.001929,121,2.257499,48.0,1.569174,4,6.868676,0,0,0
4,0,4,0.943478,NaN,15.543,13,18,5,0.000000,NaN,...,0.002489,477,0.648326,40.0,1.600173,5,5.299540,0,0,0


In [54]:
# Save chunk_level features (Optional)
feature_df.to_parquet(f"{FEATURE_PATH}/features.parquet")

In [55]:
# Store feature names for later use
feature_names = [col for col in feature_df.columns 
                    if col not in ['signal_id', 'chunk_id', 'target', 'id_measurement', 'phase']]

print(f"Number of feature columns: {len(feature_names)}")
    
# Show feature distribution by tier
basic_features = [f for f in feature_names if f in [
    'std', 'rms', 'mean_abs', 'min_val', 'max_val', 'peak_to_peak', 
    'crest_factor', 'form_factor', 'mean_abs_deviation', 'variance'
]]
signal_quality_features = [f for f in feature_names if f in [
    'zero_crossing_rate', 'peak_count', 'peak_density', 'envelope_mean', 
    'envelope_std', 'envelope_variation', 'high_freq_ratio'
]]
advanced_features = [f for f in feature_names if 'spectral' in f or 'wavelet' in f  or 'dominant' in f or 'autocorr' in f]
power_system_features = [f for f in feature_names if f not in basic_features + signal_quality_features ]

print(f"Basic features (Fastest): {len(basic_features)}")
print(f"Signal Quality features (fast): {len(signal_quality_features)}")
print(f"Advanced features (advanced): {len(advanced_features)}")
print(f"Power System features (advanced): {len(power_system_features)}")

Number of feature columns: 31
Basic features (Fastest): 10
Signal Quality features (fast): 7
Advanced features (advanced): 9
Power System features (advanced): 14


In [56]:
power_system_features

['spectral_centroid',
 'dominant_frequency',
 'dominant_magnitude',
 'dominant_power_ratio',
 'wavelet_approx_energy_ratio',
 'wavelet_detail_1_energy_ratio',
 'wavelet_detail_2_energy_ratio',
 'wavelet_detail_3_energy_ratio',
 'autocorr_first_min',
 'thd_estimate',
 'fundamental_frequency',
 'transient_index',
 'max_gradient',
 'amplitude_variation']

#### Aggregating Chunk Level Features to Signal Level

In [57]:
# Group by signal_id
grouped = feature_df.groupby('signal_id')
aggregation_methods=['mean', 'std', 'min', 'max']

aggregated_features = []

for signal_id, group in grouped:
    agg_dict = {'signal_id': signal_id}
    
    # Copy metadata columns 
    metadata_cols = ['target', 'id_measurement', 'phase']
    for col in metadata_cols:
        if col in group.columns:
            agg_dict[col] = group[col].iloc[0]
    
    # Aggregate numerical features
    numerical_cols = [col for col in feature_names if group[col].dtype in ['float64', 'int64']]
    
    for col in numerical_cols:
        for method in aggregation_methods:
            if method == 'mean':
                agg_dict[f'{col}_mean'] = group[col].mean()
            elif method == 'std':
                agg_dict[f'{col}_std'] = group[col].std()
            elif method == 'min':
                agg_dict[f'{col}_min'] = group[col].min()
            elif method == 'max':
                agg_dict[f'{col}_max'] = group[col].max()
            elif method == 'median':
                agg_dict[f'{col}_median'] = group[col].median()
            elif method == 'q25':
                agg_dict[f'{col}_q25'] = group[col].quantile(0.25)
            elif method == 'q75':
                agg_dict[f'{col}_q75'] = group[col].quantile(0.75)
    
    aggregated_features.append(agg_dict)

result_df = pd.DataFrame(aggregated_features)
result_df.shape

(5000, 128)

In [58]:
result_df.head()

,signal_id,target,id_measurement,phase,std_mean,std_std,std_min,std_max,rms_mean,rms_std,...,transient_index_min,transient_index_max,max_gradient_mean,max_gradient_std,max_gradient_min,max_gradient_max,amplitude_variation_mean,amplitude_variation_std,amplitude_variation_min,amplitude_variation_max
0,0,0,0,0,0.969092,0.049162,0.860242,1.080451,7.341025,1.681216,...,1.463640,1.655748,5.933333,2.563543,4,16,6.818389,2.038527,4.989672,15.518804
1,1,0,0,1,0.942165,0.043680,0.836421,1.039576,7.142753,1.804649,...,1.432772,1.630322,4.316667,0.469102,4,5,5.909052,0.668041,4.204635,7.140720
2,10,0,3,1,1.731215,0.848746,1.219666,7.069020,3.508683,2.074712,...,0.282088,1.609108,5.066667,12.727745,3,101,5.734389,2.919719,-15.560856,10.362359
3,100,0,33,1,1.197569,0.068980,1.043908,1.349932,6.416758,2.425201,...,0.877135,1.467941,3.900000,0.656131,3,6,5.756585,0.511805,4.741198,6.889594
4,1000,0,333,1,0.706477,0.038105,0.638905,0.804366,5.061745,1.742850,...,0.845962,1.013621,2.983333,0.833446,2,7,7.578189,1.153090,5.407796,13.675374


In [59]:
# Saving the signal granulairty features (Optional)
result_df.to_parquet(f"{FEATURE_PATH}/agg_features.parquet")

In [60]:
result_df.columns

Index(['signal_id', 'target', 'id_measurement', 'phase', 'std_mean', 'std_std',
       'std_min', 'std_max', 'rms_mean', 'rms_std',
       ...
       'transient_index_min', 'transient_index_max', 'max_gradient_mean',
       'max_gradient_std', 'max_gradient_min', 'max_gradient_max',
       'amplitude_variation_mean', 'amplitude_variation_std',
       'amplitude_variation_min', 'amplitude_variation_max'],
      dtype='object', length=128)

### Feature Preprocessing 

In [61]:
processed_df = result_df.copy()
preprocessing_info = {}

# Separate feature columns from metadata
metadata_cols = ['signal_id', 'chunk_id', 'target', 'id_measurement', 'phase']
feature_cols = [col for col in processed_df.columns if col not in metadata_cols]

#### Missing Values

In [62]:
# Handle missing values
processed_df[feature_cols] = processed_df[feature_cols].fillna(processed_df[feature_cols].median())

preprocessing_info['missing_values_handled'] = 'Median'
preprocessing_info['original_feature_count'] = len(feature_cols)

#### Infinite Values

In [63]:
# Handle infinite values
processed_df[feature_cols] = processed_df[feature_cols].replace([np.inf, -np.inf], np.nan)
processed_df[feature_cols] = processed_df[feature_cols].fillna(processed_df[feature_cols].median())

#### Select High Variance features

In [64]:
# Remove low variance features
variance_threshold=0.01
from sklearn.feature_selection import VarianceThreshold
feature_selector = VarianceThreshold(threshold=variance_threshold)

# Fit on feature columns only
feature_data = processed_df[feature_cols]
selected_features = feature_selector.fit_transform(feature_data)

# Get selected feature names
selected_feature_names = [feature_cols[i] for i in range(len(feature_cols)) 
                        if feature_selector.variances_[i] > variance_threshold]

# Update feature columns
feature_cols = selected_feature_names

# Reconstruct dataframe
temp_df = processed_df.copy()
for i, col in enumerate(selected_feature_names):
    temp_df[col] = selected_features[:, i]
processed_df = temp_df

preprocessing_info['features_after_variance_selection'] = len(feature_cols)
preprocessing_info['removed_low_variance_features'] = preprocessing_info['original_feature_count'] - len(feature_cols)


#### Feature Scaling

In [65]:
# Scale features
scaler = RobustScaler()
scaled_features = scaler.fit_transform(processed_df[feature_cols])
    
# Update dataframe with scaled features
temp_df = processed_df.copy()
for i, col in enumerate(feature_cols):
    temp_df[col] = scaled_features[:, i]
processed_df = temp_df

In [66]:
preprocessing_info['scaler_type'] = 'median'
preprocessing_info['final_feature_count'] = len(feature_cols)
preprocessing_info['final_feature_names'] = feature_cols

print(f"Preprocessing complete:")
print(f"  Original features: {preprocessing_info['original_feature_count']}")
print(f"  After variance selection: {preprocessing_info['features_after_variance_selection']}")
print(f"  Final features: {preprocessing_info['final_feature_count']}")

Preprocessing complete:
  Original features: 124
  After variance selection: 92
  Final features: 92


#### Saving Final Features for modelling

In [67]:
processed_df.to_parquet(f"{FEATURE_PATH}/final_features.parquet")

### End of Feature Engineering